In [5]:
# Basic imports
import numpy as np
import xarray as xr
import xdatasets as xd
from scipy.stats import genextreme, genpareto, gumbel_r

import xhydro as xh

try:
    from xhydro.extreme_value_analysis.parameterestimation import *

    has_julia = True
except ImportError:
    print(
        "This notebook requires the `julia` programming language and the `juliacall` Python package to be installed."
    )

    has_julia = False

In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import warnings

import pandas as pd
import pooch

# Basic imports
from xhydro.extreme_value_analysis.parameterestimation import *

warnings.simplefilter("always", UserWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
GITHUB_URL = "https://github.com/hydrologie/xhydro-testdata"
# TODO: branch will become 'main' once PR https://github.com/hydrologie/xhydro-testdata/pull/15 is merged
BRANCH_OR_COMMIT_HASH = "extreme_value_analysis"

data = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/extreme_value_analysis/NOAA_GFDL_ESM4.zip",
    known_hash="md5:6c0f51c59add2f37b0e4af169b7eb7f6",
    processor=pooch.Unzip(),
)

df = pd.read_csv(data[0])
ds = df.to_xarray()
ds = ds.set_coords(
    ["time", "station_num", "station_name", "qmax_yearly", "max_temp_yearly"]
)
ds = ds.set_index(index=["station_num", "time"])
ds = ds.unstack("index")
ds

C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\julia_import.py:20: UserWarning: juliacall module already imported. Make sure that you have set the environment variable `PYTHON_JULIACALL_HANDLE_SIGNALS=yes` to avoid segfaults. Also note that xhydro will not be able to configure `PYTHON_JULIACALL_THREADS` or `PYTHON_JULIACALL_OPTLEVEL` for you.


<xarray.Dataset> Size: 455kB
Dimensions:          (station_num: 97, time: 146)
Coordinates:
  * station_num      (station_num) int64 776B 1001 1004 1008 ... 15403 15414
  * time             (time) object 1kB '1955-01-01' ... '2100-01-01'
    station_name     (station_num, time) object 113kB 'Dozois' ... 'Romaine 4...
    qmax_yearly      (station_num, time) float64 113kB 424.0 805.1 ... 711.7
    max_temp_yearly  (station_num, time) float64 113kB 8.0 8.1 8.1 ... 4.4 4.6
Data variables:
    total_precip     (station_num, time) float64 113kB 983.6 ... 1.065e+03

In [27]:
if has_julia:
    params = fit(ds_4fa, dist=genextreme, method="ml")
    params

C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\structures\util.py:99: UserWarning: There was an error in computing confidence interval. Returned parameter and confidence interval values are numpy.nan
┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:20
C:\Users\HP\Desktop\Stage\xhydro\src\xhydro\extreme_value_analysis\structures\util.py:99: UserWarning: There was an error in computing confidence interval. Returned parameter and confidence interval values are numpy.nan
┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:2

<xarray.Dataset> Size: 15kB
Dimensions:             (station_num: 97, dparams: 6)
Coordinates:
  * station_num         (station_num) int64 776B 1001 1004 1008 ... 15403 15414
  * dparams             (dparams) <U31 744B 'scale' ... 'shape_qmax_yearly_co...
Data variables:
    total_precip        (station_num, dparams) float64 5kB nan nan ... nan nan
    total_precip_lower  (station_num, dparams) float64 5kB nan nan ... nan nan
    total_precip_upper  (station_num, dparams) float64 5kB nan nan ... nan nan
Attributes:
    long_name:    genpareto parameters
    description:  Parameters of the genpareto distribution
    method:       ML
    estimator:    Maximum likelihood
    scipy_dist:   genpareto
    units:        
    history:      [2024-08-15 14:42:34] fit: Estimate distribution parameters...

┌ Warning: The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Extremes C:\Users\HP\.julia\packages\Extremes\4Tbee\src\parameterestimation\maximumlikelihood.jl:20
